In [1]:
!pip install -U --quiet langchain-community tiktoken langchain-openai langchainhub chromadb langchain langgraph langchain-text-splitters

You should consider upgrading via the '/Users/aryankeshri/Downloads/Entivin_Task/lang_graph_rag/bin/python3 -m pip install --upgrade pip' command.


In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key")
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "multihop-rag"

OpenAI API Key········


In [2]:
from langchain.callbacks.tracers import LangChainTracer
# LANGCHAIN_TRACING_V2=True
# LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
# LANGCHAIN_API_KEY="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"
LANGCHAIN_PROJECT="multihop-rag"
tracer = LangChainTracer()
tracer.run_name = "multihop-rag"

In [21]:
from langchain_community.document_loaders import PyPDFLoader
file_path = "/Users/aryankeshri/Downloads/Entivin_Task/Resources/Multihop_QA.pdf"
loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [12]:
print(pages)
# print(f"{pages[4].metadata}\n")
# print(pages[4].page_content)

In [43]:
from langchain.schema import Document
import json

with open('./hotpot_dev_distractor_v1.json', 'r') as fp:
    dataset = json.load(fp)
print(len(dataset))
print(round(0.40*len(dataset)))
dataset = dataset[:round(0.40*len(dataset))]
# # print(f"new_length: {len(dataset)}")
keys_pop = ["answer", "question"]

qa_dict = []

for d in dataset:
    quest = d["question"]
    ans = d["answer"]
    
    qa_dict.append({
        "id": d["_id"],
        "question": [quest],
        "answer": ans
    })
    
    for key in keys_pop:
        d.pop(key, None)
# print(qa_dict)

7405
2962


## Create Index

In [45]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import json


# Create a Document object
# include metadata
documents = []
print(len(dataset))
for item in dataset:
    # Combine context sections into a single string for better embeddings
    context = item.get("context", [])
    page_content = ""
    for section in context:
        section_title = section[0]
        section_texts = section[1]
        # Concatenate title and its corresponding texts
        page_content += f"{section_title}: " + " ".join(section_texts) + "\n"

    # Create a Document object with the combined context and _id as metadata
    doc = Document(
        page_content=page_content,
        metadata={
            "_id": item["_id"]
        }
    )
    documents.append(doc)
print(len(documents))

2962
2962


In [40]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# Function to calculate token usage
def calculate_total_tokens(documents, chunk_size):
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=chunk_size, chunk_overlap=0
    )
    token_count = 0
    for doc in documents:
        chunks = text_splitter.split_text(doc.page_content)
        token_count += sum(len(chunk.split()) for chunk in chunks)  # Approximate token count
    return token_count

# Initial dataset size
original_tokens = calculate_total_tokens(documents, chunk_size=250)
print(f"Original token count: {original_tokens}")

# Reduce dataset until it's within the token limit
while original_tokens > 1_000_000:
    documents = documents[:int(len(documents) * 0.9)]  # Reduce by 10% iteratively
    original_tokens = calculate_total_tokens(documents, chunk_size=250)

print(f"Final token count: {original_tokens}")
print(f"Final dataset size: {len(documents)}")

Original token count: 2747347
Final token count: 949112
Final dataset size: 1029


In [47]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI
import time

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=50
)

# Split documents into chunks
chunks = []
for idx, doc in enumerate(documents):
    chunked_texts = text_splitter.split_text(doc)
    for chunk in chunked_texts:
        # Create Document object for each chunk with metadata
        chunks.append(Document(page_content=chunk, metadata={"source_doc_id": idx}))

# Monitor token usage
total_tokens = sum(len(chunk.page_content.split()) for chunk in chunks)
print(f"Total tokens in dataset: {total_tokens}")

# Initialize ChatOpenAI embeddings
embedding_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Initialize the Chroma vector store
vectorstore = Chroma(collection_name="chat-chroma", embedding_function=embedding_model)

# Batch size for processing
batch_size = 50  # Number of chunks per batch

# Process and add documents to vector store in batches
for i in range(0, len(chunks), batch_size):
    batch = chunks[i:i + batch_size]

    # Extract content and metadata for embeddings
    batch_texts = [doc.page_content for doc in batch]
    batch_metadatas = [doc.metadata for doc in batch]

    # Generate embeddings and add to vector store
    vectorstore.add_texts(texts=batch_texts, metadatas=batch_metadatas)
    
    # Sleep to avoid exceeding rate limits (adjust as needed)
    time.sleep(60)

print("All documents have been successfully added to the vector store.")

TypeError: expected string or bytes-like object

## LLM

In [4]:
### Retrieval Grader

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI


# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
# question = "What are Multihop RAG?"
question = "What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?"
docs = retriever.get_relevant_documents(question)
print(docs)
doc_txt = docs[0].page_content
# doc_txt = doc.context[]
print(doc_txt)
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

/Users/aryankeshri/Downloads/Entivin_Task/lang_graph_rag/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
/var/folders/_4/ky3k7xbn09562xrb4g_gkbsc0000gn/T/ipykernel_69167/1919133848.py:35: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.g

[Document(metadata={'_id': '5a8c7595554299585d9e36b6'}, page_content='{\n    "_id": "5a8c7595554299585d9e36b6",\n    "supporting_facts": [\n        [\n            "Kiss and Tell (1945 film)",\n            0\n        ],\n        [\n            "Shirley Temple",\n            0\n        ],\n        [\n            "Shirley Temple",\n            1\n        ]\n    ],\n    "context": [\n        [\n            "Meet Corliss Archer",\n            [\n                "Meet Corliss Archer, a program from radio\'s Golden Age, ran from January 7, 1943 to September 30, 1956.",\n                "Although it was CBS\'s answer to NBC\'s popular \\"A Date with Judy\\", it was also broadcast by NBC in 1948 as a summer replacement for \\"The Bob Hope Show\\".",\n                "From October 3, 1952 to June 26, 1953, it aired on ABC, finally returning to CBS.",\n                "Despite the program\'s long run, fewer than 24 episodes are known to exist."\n            ]\n        ],\n        [\n            "

In [5]:
### Generate

from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

Shirley Temple held the government position of United States ambassador to Ghana and to Czechoslovakia, as well as Chief of Protocol of the United States.


## Make tool

make tool for your multihop rag here

In [6]:
### Question Decomposer

# LLM
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

# Prompt
system = """You are a question decomposer that decomposes a question into 2 or 3 sub-questions, if the original question is complex or indirect or not straightforward.\n
Look at the input and try to reason about the underlying semantic intent / meaning."""

requery_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Decompose into two or three sub-question as per the requirement."
        )
    ]
)

question_decomposer = requery_prompt | llm | StrOutputParser()
question_decomposer.invoke({"question": question})

'1. Who portrayed Corliss Archer in the film Kiss and Tell?\n2. What government position did the woman who portrayed Corliss Archer hold?'

## Create graphs

Now let's create our graph for implementing multi-hop features

### Define Graph State

In [7]:
from typing import List, Set
from typing_extensions import TypedDict

class GraphState(TypedDict):
    """
    Represents the state of our graph. 
    
    Attributes:
        question: question
        generation: LLM generation
        iterative_Search: whether to add search
        documents: list of documents
    """
    
    questions: List[str]
    generation: str
    requery_search: str
    documents: List[str]

In [8]:
from langchain.schema import Document
from tqdm.auto import tqdm

def retrieve(state):
    """
    Retrieve documents
    
    Args: 
        state (dict): The current graph state
        
    Returns: 
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    
    print("--Retrieve--\n")
    documents = []
    questions = state['questions']
    
    for question in questions:
        # Retrieval 
        documents.extend(retriever.get_relevant_documents(question))
        
    return {"documents": documents, "questions": questions}

def generate(state):
    """
    Generate Answer: 
    
    Args:
        state (dict): The current graph state
        
    Returns: 
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---Generate---\n")
    
    questions = state['questions']
    documents = state['documents']
    generation = ""
    # RAG generation
    print(documents)
    for question in questions:
        generation = generation + rag_chain.invoke({'context': documents, 'question': question})
        
    return {"documents": documents, "questions": questions[0], "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.
    
    Args: 
        state (dict): The current graph state
    
    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """
    
    print("---Check Document Relevance to Question---\n")
    question = state["questions"]
    documents = state["documents"]
    print(documents)
    # Score each doc
    filtered_docs = []
    re_query = "No"
    for d in documents: # Its a single loop cause, in the begining there is only one question in the list. and this function is called only in the begining

        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---\n")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---\n")
            re_query = "Yes"
            continue
#         else:
#             print("---No relevant document found---\n")
#             re_query = "Yes"
    return {"documents": filtered_docs, "questions": question, "requery_search": re_query}


def decompose_query(state):
    """
    Decompose the query into sub-queries for better retrieval.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appends multiple questions to questions key with a re-phrased question
    """
    print("---Decompose Query---\n")
    question = state['questions'][0]
    
    # Decomposed questions 
    questions_str = question_decomposer.invoke({"question": question})
    print(questions_str)
    print("\n")
    new_questions = questions_str.split('\n')
    
    return {"documents": documents, "questions": new_questions}

def decide_requery(state):
    """
    Determines whether to generate an answer, or decompose a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """
    print("---Assesing Graded Documents---\n")
    state['questions']
    requery = state['requery_search']
    state['documents']
    
    if requery == "Yes":
        print("---Decomposing Question into sub-questions---\n")
        return "decompose_query"
    else:
        print("---Decision Generate---\n")
        return "generate"

In [9]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve) # retrieve
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("decompose_query", decompose_query)
workflow.add_node("retrieve_after_decomposition", retrieve)
workflow.add_node("generate", generate)
# workflow.add_node("")

# Build graph
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_requery,
    {
        "decompose_query": "decompose_query",
        "generate": "generate"
    },
)
workflow.add_edge("decompose_query", "retrieve_after_decomposition")
workflow.add_edge("retrieve_after_decomposition", "generate")
workflow.add_edge("generate", END)

# Compile
app = workflow.compile()

## Use the Graph

In [11]:
from pprint import pprint

# Run
inputs = {"questions": ["What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?"]}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

--Retrieve--



Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


"Node 'retrieve':"
'\n---\n'
---Check Document Relevance to Question---

[Document(metadata={'_id': '5a8c7595554299585d9e36b6'}, page_content='{\n    "_id": "5a8c7595554299585d9e36b6",\n    "supporting_facts": [\n        [\n            "Kiss and Tell (1945 film)",\n            0\n        ],\n        [\n            "Shirley Temple",\n            0\n        ],\n        [\n            "Shirley Temple",\n            1\n        ]\n    ],\n    "context": [\n        [\n            "Meet Corliss Archer",\n            [\n                "Meet Corliss Archer, a program from radio\'s Golden Age, ran from January 7, 1943 to September 30, 1956.",\n                "Although it was CBS\'s answer to NBC\'s popular \\"A Date with Judy\\", it was also broadcast by NBC in 1948 as a summer replacement for \\"The Bob Hope Show\\".",\n                "From October 3, 1952 to June 26, 1953, it aired on ABC, finally returning to CBS.",\n                "Despite the program\'s long run, fewer than 24 episodes 